In [12]:
import os
import pandas as pd
import json
import numpy as np

#data files
path_to_capacity_over_time = "./data_wrangling_2/raw_data/capacity_over_time.xlsx"
path_to_data_ticker = "./data_wrangling_2/raw_data/data_ticker.xlsx"
path_to_gap_in_ambition = "./data_wrangling_2/raw_data/gap_in_ambition.xlsx"
path_to_past_and_planned_retirement = "./data_wrangling_2/raw_data/past_and_planned_retirements.xlsx"
path_to_planned_retire_groupings = "./data_wrangling_2/raw_data/planned_retire_groupings.xlsx"

#file name headers
project_data_dir = "./data"
year_filename = "h2_2024"
full_year = "2024"
year_variable = "H2 2024"

# Print the current working directory
print(f"Current working directory: {os.getcwd()}")

# Optionally, change the working directory if needed
# os.chdir('/path/to/correct/directory')  # Uncomment this line if needed

# Define the correct path using absolute or relative paths
path_to_public_assets_folder = os.path.join(os.getcwd(), "public/assets")

print(f"Checking public assets folder path: {repr(path_to_public_assets_folder)}")

if os.path.isdir(path_to_public_assets_folder):
    print("Public assets folder exists!")
    path_to_data_folder = os.path.join(path_to_public_assets_folder, "data")
    
    # Check and create the data folder if necessary
    if not os.path.exists(path_to_data_folder):
        os.makedirs(path_to_data_folder)
        print(f"Creating data folder at: {path_to_data_folder}")
    else:
        print(f"Data folder already exists at: {path_to_data_folder}")
else:
    print(f"Public assets folder does not exist at: {path_to_public_assets_folder}")

Current working directory: /Users/christines/Desktop/workspace/Phaseout-dashboard
Checking public assets folder path: '/Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets'
Public assets folder exists!
Data folder already exists at: /Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets/data


In [13]:
# Function to generate the json files

def pandas_df_to_json(
    dataframe: pd.DataFrame,
    output_path: str,
):
    our_json_file = []
    for idx, row in dataframe.iterrows():
        curr_dict = {}
        for column in dataframe.columns:
            curr_dict[column] = str(row[column])
        
        our_json_file.append(curr_dict)
    
    with open(output_path, "w") as output:
        json.dump(our_json_file, output, indent=4) 
        
    print(f"Wrote json file to {output_path}")

In [14]:
# 1. Capacity over time
capacity_over_time = pd.read_excel(path_to_capacity_over_time)
capacity_over_time = capacity_over_time.fillna("")
#capacity_over_time = capacity_over_time.fillna(0)

pandas_df_to_json(
    capacity_over_time, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_capacity_over_time.json")
)

Wrote json file to /Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets/data/gcpt_h2_2024_capacity_over_time.json


In [28]:
# 2. Data ticker

data_ticker = pd.read_excel(path_to_data_ticker)

data_ticker["Pledged coal phaseout year"] = data_ticker["Pledged coal phaseout year"].fillna("None")
data_ticker["Pledged coal phaseout year"] = data_ticker["Pledged coal phaseout year"].astype(str).str.replace(".0", "")

for idx, row in data_ticker.iterrows():
    pledged_coal_phaseout_year = row["Pledged coal phaseout year"]
    formatted_pledged_coal_phaseout_year = f"<span>{pledged_coal_phaseout_year}</span><br>Pledged Coal<br>Phaseout Year"
    data_ticker.loc[idx, "Pledged coal phaseout year"] = formatted_pledged_coal_phaseout_year
    
    net_zero_coal_phaseout_year = row["1.5C pathway phaseout year"]
    formatted_net_zero_coal_phaseout_year= f"<span>{net_zero_coal_phaseout_year}</span><br>1.5C Pathway<br>Phaseout Year"
    data_ticker.loc[idx, "1.5C pathway phaseout year"] = formatted_net_zero_coal_phaseout_year
    
    total_operating_capacity = row["Total operating capacity (GW)"]
    formatted_total_operating_capacity= f"<span>{{{{{total_operating_capacity}}}}}</span><br>Total Operating<br>Capacity (GW)"
    data_ticker.loc[idx, "Total operating capacity (GW)"] = formatted_total_operating_capacity


data_ticker.rename(
    {
        "Pledged coal phaseout year": "summary_1",
        "1.5C pathway phaseout year": "summary_2",
        "Total operating capacity (GW)": "summary_3"
    },
    axis=1,
    inplace=True
)

data_ticker["summary_1_color"] = "#bf532c"
data_ticker["summary_2_color"] = "#A8A8A8"
data_ticker["summary_3_color"] = "#f27d16"

data_ticker = data_ticker[["Country", "summary_1", "summary_1_color", "summary_2", "summary_2_color", "summary_3", "summary_3_color"]]

pandas_df_to_json(
    data_ticker, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_data_ticker.json")
)

Wrote json file to /Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets/data/gcpt_h2_2024_data_ticker.json


In [27]:
# 3. Gap in ambition
gap_in_ambition = pd.read_excel(path_to_gap_in_ambition)

for idx, row in gap_in_ambition.iterrows():
    value = row["Capacity"]
    if value >= 10:
        new_value = np.round(value, 0)  # Round to 0 decimals for values >= 10
    elif 1 <= value < 10:
        new_value = np.round(value, 1)  # Round to 1 decimal for values between 1 and 10
    elif value < 1:  # Handle all values less than 1 (including 0)
        new_value = np.round(value, 2)  # Round to 2 decimals for values less than 1
    gap_in_ambition.loc[idx, "Capacity"] = new_value

pandas_df_to_json(
    gap_in_ambition, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_gap_in_ambition.json")
)

Wrote json file to /Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets/data/gcpt_h2_2024_gap_in_ambition.json


In [22]:
# 4. path_to_past_and_planned_retirement
past_and_planned_retirement = pd.read_excel(path_to_past_and_planned_retirement)
pandas_df_to_json(
    past_and_planned_retirement, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_past_and_planned_retirement.json")
)

Wrote json file to /Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets/data/gcpt_h2_2024_past_and_planned_retirement.json


In [23]:
# 5. path_to_planned_retire_groupings
planned_retire_groupings = pd.read_excel(path_to_planned_retire_groupings)

pandas_df_to_json(
    planned_retire_groupings, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_planned_retire_groupings.json")
)

Wrote json file to /Users/christines/Desktop/workspace/Phaseout-dashboard/public/assets/data/gcpt_h2_2024_planned_retire_groupings.json


In [ ]:
### DELETE THIS IF THE ABOVE WORKS ###

path_to_capacity_over_time = "./data_wrangling_2/raw_data/capacity_over_time.xlsx"
path_to_data_ticker = "./data_wrangling_2/raw_data/data_ticker.xlsx"
path_to_gap_in_ambition = "./data_wrangling_2/raw_data/gap_in_ambition.xlsx"
path_to_past_and_planned_retirement = "./data_wrangling_2/raw_data/past_and_planned_retirements.xlsx"
path_to_planned_retire_groupings = "./data_wrangling_2/raw_data/planned_retire_groupings.xlsx"


#file name headers
project_data_dir = "./data"
year_filename = "h2_2024"
full_year = "2024"
year_variable = "H2 2024"

# Define where the public folder is.
path_to_public_assets_folder = "/Users/christineshearer/Desktop/workspace/Phaseout-dashboard/public/assets"

if os.path.exists(path_to_public_assets_folder):
    
    # Define where the data folder is.
    path_to_data_folder = os.path.join(path_to_public_assets_folder, "data")
    
    # If the folder does not exist yet, create it.
    if not os.path.exists(path_to_data_folder):
        os.mkdir(path_to_data_folder)
        print(f"Creating data folder at: {path_to_data_folder}")

In [ ]:
### Delete this if #3 above has no issues

# 3. Gap in ambition
gap_in_ambition = pd.read_excel(path_to_gap_in_ambition)

for idx, row in gap_in_ambition.iterrows():
    value = row["Capacity"]
    if value >= 10:
        new_value = np.round(value, 0)
    elif 1 <= value < 10:
        new_value = np.round(value, 1)
    elif 0 < value < 1:
        new_value = np.round(value, 2)
    elif value == 0:  # Add this case for zero
        new_value = 0  # Handle zero explicitly
    gap_in_ambition.loc[idx, "Capacity"] = new_value

     
pandas_df_to_json(
    gap_in_ambition, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_gap_in_ambition.json")
)